In [1]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 38.6 MB/s eta 0:00:00


In [3]:
import fitz  # PyMuPDF for PDF text extraction
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

In [4]:
# Load Hugging Face QA Model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Load Sentence Transformer Model for Embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = [page.get_text("text") for page in doc]  # Extract text per page
    return text  # Returns a list of pages (chunked naturally)

In [ ]:
[1,15,16,17]

[1,20,16,17], [1,13,16,17]

In [7]:
# Function to get the most relevant text chunk using embeddings & cosine similarity
def find_most_relevant_chunk(question, text_chunks):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)
    chunk_embeddings = [embedding_model.encode(chunk, convert_to_tensor=True) for chunk in text_chunks]

    # Compute cosine similarity
    similarities = [util.pytorch_cos_sim(question_embedding, chunk)[0].item() for chunk in chunk_embeddings]

    # Find the most relevant chunk (highest similarity)
    best_chunk_idx = similarities.index(max(similarities))
    return text_chunks[best_chunk_idx]

In [8]:
# Main QA Function
def answer_question_from_pdf(pdf_path, question):
    text_chunks = extract_text_from_pdf(pdf_path)  # Extract & split text
    relevant_chunk = find_most_relevant_chunk(question, text_chunks)  # Find best chunk

    # Use QA Model to extract answer
    result = qa_pipeline(question=question, context=relevant_chunk)
    return result["answer"]

In [10]:
# Example Usage
pdf_path = "/content/Resume_Lakshmi_UpdatedF.pdf"  # Replace with your actual PDF file
question = input("Enter your question: ")  # Get user input
answer = answer_question_from_pdf(pdf_path, question)
print(f"Answer: {answer}")


Enter your question: From which university did the candidate pursue M.tech
Answer: 
RV University
